In [1]:
import pandas as pd
import datetime as datetime

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
gol_games_df = pd.read_csv('../../post_champ_selection/data/MATCH_SUMMARY_2024.csv')
gol_games_df.head()


,gol_game_num,league,season,year,date,week,patch,format,game_in_format,game_length(min),game_length(s),red_team,blue_team,red_team_outcome,blue_team_outcome,winner,mvp,season_format,loser
0,57867,LPL,SPRING,2024,2024-04-20,FINALS,14.6,BO5,1,43:17,2597,Top Esports,Bilibili Gaming,LOSS,WIN,Bilibili Gaming,NaN,PLAYOFFS,Top Esports
1,57868,LPL,SPRING,2024,2024-04-20,FINALS,14.6,BO5,2,23:12,1392,Bilibili Gaming,Top Esports,WIN,LOSS,Bilibili Gaming,NaN,PLAYOFFS,Top Esports
2,57869,LPL,SPRING,2024,2024-04-20,FINALS,14.6,BO5,3,36:43,2203,Bilibili Gaming,Top Esports,LOSS,WIN,Top Esports,NaN,PLAYOFFS,Bilibili Gaming
3,57870,LPL,SPRING,2024,2024-04-20,FINALS,14.6,BO5,4,32:40,1960,Top Esports,Bilibili Gaming,LOSS,WIN,Bilibili Gaming,NaN,PLAYOFFS,Top Esports
4,57844,LPL,SPRING,2024,2024-04-14,SEMIFINALS,14.6,BO5,1,29:34,1774,JD Gaming,Top Esports,LOSS,WIN,Top Esports,NaN,PLAYOFFS,JD Gaming


In [4]:
gol_games_df.week.unique()

array(['FINALS', 'SEMIFINALS', 'ROUND4', 'ROUND3', 'ROUND2', 'ROUND1',
       'WEEK9', 'WEEK8', 'WEEK7', 'WEEK6', 'WEEK5', 'WEEK4', 'WEEK3',
       'WEEK2', 'WEEK1', 'TIEBREAKERS'], dtype=object)

In [5]:
lol_game_df = pd.read_csv('../data/event.csv')
lol_game_df.head()

,startTime,blockName,league,match_id,type,team_1_name,team_1_code,team_1_result,team_2_name,team_2_code,team_2_result,format,games
0,2024-03-09T16:00:00Z,Week 1,LEC,111997906552170246,normal,Team Vitality,VIT,1,Team Heretics,TH,0,bestOf_1,"[['111997906552170247', [{'parameter': 'JlLMX8..."
1,2024-03-09T17:00:00Z,Week 1,LEC,111997906552104652,normal,Fnatic,FNC,1,Rogue,RGE,0,bestOf_1,"[['111997906552104653', [{'parameter': 'u7u4P3..."
2,2024-03-09T18:00:00Z,Week 1,LEC,111997906552104656,normal,Team BDS,BDS,0,G2 Esports,G2,1,bestOf_1,"[['111997906552104657', [{'parameter': 'vA1IMd..."
3,2024-03-09T19:00:00Z,Week 1,LEC,111997906552104672,normal,Karmine Corp,KC,1,GIANTX,GX,0,bestOf_1,"[['111997906552104673', [{'parameter': '6P57zZ..."
4,2024-03-09T20:00:00Z,Week 1,LEC,111997906552170250,normal,MAD Lions KOI,MDK,1,SK Gaming,SK,0,bestOf_1,"[['111997906552170251', [{'parameter': 'qDNqLb..."


In [6]:
lol_game_df.type.unique()

array(['normal', 'tiebreaker'], dtype=object)

In [7]:
gol_games_df = pd.DataFrame(gol_games_df[gol_games_df['league'] != 'LPL'].reset_index())
_ = gol_games_df.pop('mvp')
gol_games_df['date'] = pd.to_datetime(gol_games_df['date']).dt.date

lol_game_df['startTime'] = pd.to_datetime(lol_game_df['startTime'])


In [8]:
gol_games_df['red_team'].unique()
print(len(gol_games_df['red_team'].unique()))

lol_game_df['team_1_name'].unique()
print(len(lol_game_df['team_1_name'].unique()))

for team in lol_game_df['team_1_name'].unique():
    if team not in set(gol_games_df['red_team'].unique()):
        print(team)                       

for team in gol_games_df['red_team'].unique():
    if team not in set(lol_game_df['team_1_name'].unique()):
        print(team)           

28
28


In [9]:
set(lol_game_df['team_1_name'].unique()).issubset(set(gol_games_df['red_team'].unique()))

True

In [10]:
gol_games_df['riot_match_id'] = ''
gol_games_df['min_start_time'] = ''


In [11]:
t1_outcome = 'LOSS' if lol_game_df.iloc[2]['team_1_result'] == 0 else 'WIN'
t1_outcome

'LOSS'

In [12]:
seen_mri = set()
for index, row in lol_game_df.iterrows():
    # if row['match_id'] != 111861082639032027:
    #     continue
    
    match_indexes = []
    base_match_id = row['match_id']
    base_start_time = row['startTime']
    riot_patch = row['startTime']


    conv_date = row['startTime']
    if conv_date.time() == datetime.time(0, 0):
        conv_date = conv_date - pd.to_timedelta(1, unit='d')
    conv_date = conv_date.date()

    t1_outcome = 'LOSS' if row['team_1_result'] == 0 else 'WIN'
    t2_outcome = 'LOSS' if row['team_2_result'] == 0 else 'WIN'
    
    matching_row = gol_games_df[((gol_games_df['red_team'] == row['team_1_name']) & (gol_games_df['blue_team'] == row['team_2_name'])
                                  & (gol_games_df['date'] == conv_date))]
                                #   & (gol_games_df['red_team_outcome'] == t1_outcome) & (gol_games_df['blue_team_outcome'] == t2_outcome) )]
    match_indexes.extend(matching_row.index)
    
    matching_row = gol_games_df[((gol_games_df['blue_team'] == row['team_1_name']) & (gol_games_df['red_team'] == row['team_2_name'])
                                  & (gol_games_df['date'] == conv_date))]
                                #   & (gol_games_df['blue_team_outcome'] == t1_outcome) & (gol_games_df['red_team_outcome'] == t2_outcome) )]
    match_indexes.extend(matching_row.index)
    
    match_indexes.sort()
    time_played_s = 0
    for mri in match_indexes:
        found_row = gol_games_df.iloc[mri]
        if (found_row['week'] == 'TIEBREAKERS' and row['type'] != 'tiebreaker') or (found_row['week'] != 'TIEBREAKERS' and row['type'] == 'tiebreaker'):
            print(mri)
            print(index)
            continue
        if mri in seen_mri:
            print(mri)
            print(index)
        seen_mri.add(mri)

        
        gif = int(found_row['game_in_format'])
        gol_games_df.at[mri,'riot_match_id'] = base_match_id + gif
        if gif == 1:
            gol_games_df.at[mri,'min_start_time'] = base_start_time
        else:
            gol_games_df.at[mri,'min_start_time'] = base_start_time + pd.to_timedelta(time_played_s, unit='s')
                
        time_played_s = time_played_s + found_row['game_length(s)']

456
41
460
45


In [ ]:
gol_games_df.isnull().values.any()

In [ ]:
gol_games_df

In [13]:
gol_games_df[gol_games_df.duplicated(['riot_match_id'], keep=False)]

,index,gol_game_num,league,season,year,date,week,patch,format,game_in_format,...,game_length(s),red_team,blue_team,red_team_outcome,blue_team_outcome,winner,season_format,loser,riot_match_id,min_start_time


In [18]:
gol_games_df.to_csv('../data/gol_with_match_ids.csv',index=False)

In [ ]:
print(gol_games_df.iloc[270])

In [ ]:
print(gol_games_df.iloc[271])

In [ ]:
len(gol_games_df)

In [ ]:
len(lol_game_df)

In [ ]:
sum